In [ ]:
baizer::pkglib(Seurat, SingleR, celldex, tidyverse, baizer, genogamesh, yaml)

In [ ]:
sample <- snakemake@wildcards[['sample']]

# replace default configs with sample configs
config <- replace_item(snakemake@config, snakemake@config[[sample]])

In [ ]:
Pcount <- snakemake@input[['mRNA_count_dir']]
Pmtx <- str_glue('{Pcount}/filtered_feature_bc_matrix')
Pcsv <- snakemake@output[['mRNA_csv']]
Prds <- snakemake@output[['mRNA_rds']]
Pstat <- snakemake@output[['mRNA_stat']]
Pstat_dir <- snakemake@output[['stat_dir']]

Pcelltype_ref <- config[['celltype_ref']]
Nmin_cell_to_keep_gene <- config[['Nmin_cell_to_keep_gene']]
Nmin_gene_to_keep_cell <- config[['Nmin_gene_to_keep_cell']]
celltype_ref_tissue <- config[['celltype_ref_tissue']]
Ndim_umap_use <- config[['Ndim_umap_use']]

In [ ]:
# Seurat readin
Mcount <- Read10X(data.dir = Pmtx)
Mcount <- Mcount[rowSums(Mcount) > 10, colSums(Mcount) > 10]
SE <- CreateSeuratObject(as.data.frame(Mcount), project=sample, 
                         min.cells=Nmin_cell_to_keep_gene, min.features=Nmin_gene_to_keep_cell)
SE@meta.data <- SE@meta.data %>% rename(batch=orig.ident)

In [ ]:
# mitochondria umi percent
SE@meta.data[,'mt_percent'] <- PercentageFeatureSet(SE, pattern = "^mt-")

In [ ]:
# cell type
SEref <- readRDS(Pcelltype_ref)[[celltype_ref_tissue]]
SE <- SingleR_SE(SE, SEref)

In [ ]:
# reduction
SE <- reduction_SE(SE, Ndim_umap_use)

In [ ]:
# stat
Lstat <- list()

In [ ]:
# write
dir.create(dirname(Pcsv), recursive = TRUE)
write_excel_csv(SE@meta.data %>% r2c('cell'), Pcsv)
saveRDS(SE, Prds)
write_yaml(Lstat, file=Pstat)

In [ ]:
dir.create(Pstat_dir, recursive = TRUE)
file.copy(Pcsv, str_c(Pstat_dir, '/', basename(Pcsv)), overwrite=TRUE)
file.copy(Pstat, str_c(Pstat_dir, '/', basename(Pstat)), overwrite=TRUE)